decoder transformer层的linear1层（dtl1）替换为近似矩阵乘法

In [1]:
import numpy as np
import os
import sys
# 获取当前文件所在的文件夹路径
if "__file__" in globals():
    # 获取__file__变量的值
    file_path = __file__
    # 获取当前文件所在的文件夹路径
    dir_now = os.path.dirname(file_path)
else:
    # 获取当前工作目录
    dir_now = os.getcwd()
sys.path.append(dir_now)
sys.path.append(os.path.join(dir_now, '../'))
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # 防止jupyter爆内存
import matmul as mm
import math_util as mu
from NNutils import *
# import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

/shared/conda/envs/bolt/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_PLUTO
# method = METHOD_MITHRALPQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
# for method in [METHOD_MITHRAL, METHOD_PQ]:

In [3]:
linear_name = 'dtl1'
feedback_bits = 256
linear_name_full = "dx_linear1"

auto_train_change_nbits = False # 是否根据已运行的训练性能结果改变nbits自动训练，（train_sam_num取已训练的最大值）
auto_train_change_upcast = False # 是否根据已运行的训练性能结果改变upcast自动训练，（train_sam_num取已训练的最大值）

if auto_train_change_nbits:
    nbits_trained = 8
if auto_train_change_upcast:
    upcast_trained = 16
quantize_lut = False
nbits_goal = 6
upcast_goal = -1
lut_work_const = -1
if quantize_lut == False:
    nbits_goal = 0
nbits = nbits_goal # 要运行的量化比特数
upcast_every = upcast_goal # 要运行的upcast

test_sam_num = 1000 # 测试集样本数(如需修改，请同时修改下面的读取文件，现文件默认1000个样本)

if not auto_train_change_nbits and not auto_train_change_upcast:
    ncodebooks = 64 # max:64
    ncentroids = 256
    train_sam_num = 1000 # 训练集样本数
elif auto_train_change_nbits:
    param2change = "nbits"
    param_trained = nbits_trained
    param_goal = nbits_goal
    cb_ct_ntr_combinations_unique = change_param_auto_run_list(linear_name, method, feedback_bits, param2change, param_trained, param_goal, "upcast_every", 16)
    print(cb_ct_ntr_combinations_unique)
    # 遍历每个cb、ct、n_train_sam组合
    # for _, row_ref in cb_ct_ntr_combinations_unique.iterrows():
    #     ncodebooks = int(row_ref['cb'])
        # ncentroids = int(row_ref['ct'])
        # train_sam_num = int(row_ref['n_train_sam'])
elif auto_train_change_upcast:
    param2change = "upcast_every"
    param_trained = upcast_trained
    param_goal = upcast_goal
    cb_ct_ntr_combinations_unique = change_param_auto_run_list(linear_name, method, feedback_bits, param2change, param_trained, param_goal, "nbits", 8)
    print(cb_ct_ntr_combinations_unique)
    # 遍历每个cb、ct、n_train_sam组合
    # for _, row_ref in cb_ct_ntr_combinations_unique.iterrows():
    #     ncodebooks = int(row_ref['cb'])
        # ncentroids = int(row_ref['ct'])
        # train_sam_num = int(row_ref['n_train_sam'])

batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [4]:
AMM_train_dirs = get_AMM_train_dirs(linear_name, linear_name_full, method, feedback_bits, train_sam_num, test_sam_num)
create_dir(AMM_train_dirs["dir_result"])


In [5]:
dataset_prepare(AMM_train_dirs["dir_joined"], linear_name_full, feedback_bits, [train_sam_num, test_sam_num], 
                batch_size, S1 = S1_dict[linear_name])

查找以此为前缀的数据集： dx_linear1in_train_f256
没有比输入样本数更大的数据集，从样本合成新数据集
生成的数据集前缀:  dx_linear1in train
样本合并第一维前大小： (32, 32, 64)
样本合并第一维后大小： (1024, 64)
合并后数据集大小:  (3072000, 64)
intermediate_name[-3:] == out: False
查找以此为前缀的数据集： dx_linear1in_test_f256
没有比输入样本数更大的数据集，从样本合成新数据集
生成的数据集前缀:  dx_linear1in test
样本合并第一维前大小： (32, 32, 64)
样本合并第一维后大小： (1024, 64)
合并后数据集大小:  (1024000, 64)
intermediate_name[-3:] == out: False
查找以此为前缀的数据集： dx_linear1out_train_f256
没有比输入样本数更大的数据集，从样本合成新数据集
生成的数据集前缀:  dx_linear1out train
样本合并第一维前大小： (32, 32, 128)
样本合并第一维后大小： (1024, 128)
合并后数据集大小:  (3072000, 128)
intermediate_name[-3:] == out: True
查找以此为前缀的数据集： dx_linear1out_test_f256
没有比输入样本数更大的数据集，从样本合成新数据集
生成的数据集前缀:  dx_linear1out test
样本合并第一维前大小： (32, 32, 128)
样本合并第一维后大小： (1024, 128)
合并后数据集大小:  (1024000, 128)
intermediate_name[-3:] == out: True


In [6]:
if method == METHOD_PLUTO:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut, 
                        upcast_every=upcast_every, bias_path=AMM_train_dirs["biaspath"],lut_work_const=-1)
elif method == METHOD_MITHRAL:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut,
                        upcast_every=upcast_every, lut_work_const=lut_work_const)
else:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut,
                        upcast_every=upcast_every)

running method:  Mithral
X.shape:  (3072000, 64)
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    16495.793483881633
learn_multisplits(): returning loss:  1.2165498540271074
learn_multisplits(): initial loss:    40391.02118037222
learn_multisplits(): returning loss:  1.9325443105772138
learn_multisplits(): initial loss:    43029.03895728454
learn_multisplits(): returning loss:  2.087842100299895
learn_multisplits(): initial loss:    345555.4423322543
learn_multisplits(): returning loss:  16.394129342287535
learn_multisplits(): initial loss:    80581.71919248573
learn_multisplits(): returning loss:  5.541669798083603
learn_multisplits(): initial loss:    63568.134343922626
learn_multisplits(): returning loss:  2.82762308139354
learn_multisplits(): initial loss:    55166.96078488032
learn_multisplits(): returning loss:  3.2949237569700927
learn_multisplits(): initial loss:    18710.057500212188
learn_multisplits(): returning loss:  1.3335342779755592
learn

/home/hdr/pq/bolt/experiments/python/CsiTransformerAMM/../clusterize.py:2007: UserWarning: Persisting input arguments took 1.38s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  _learn_mithral_initialization(X, ncodebooks, ncentroids=ncentroids, pq_perm_algo='start', **kwargs)


X_res mse / X mse:  0.00034963572
fitting dense lstsq to X_res
  with X_enc:(3072000, 64) Y:(3072000, 64)


MemoryError: Unable to allocate 375. GiB for an array with shape (3072000, 16384) and data type float64

In [ ]:
x_test = np.load(AMM_train_dirs["dir_test"]+'/'+AMM_train_dirs["linearin_path_test"])
w_test = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["weightpath"])
bias = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["biaspath"])
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
if method == METHOD_PLUTO:
    y_out_last = y_out_matmul
else:
    y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [ ]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), 
                                                        y_out_last_re.astype(np.float32))
elif method == METHOD_MITHRAL or method == METHOD_PLUTO:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i_ql%i_nb%i_uc%i_lwc%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids, quantize_lut, nbits, upcast_every, lut_work_const)), y_out_last_re)
elif method == METHOD_PQ or method == METHOD_MITHRALPQ:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i_ql%i_nb%i_uc%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids, quantize_lut, nbits, upcast_every)), y_out_last_re)
else:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids)), y_out_last_re)

[[-0.42177775 -0.39216104 -0.48911437 ... -0.28617575  1.18143197
  -0.36507135]
 [-0.46662221 -0.41168101 -0.39882974 ... -0.26375007  1.1454931
  -0.3328296 ]
 [-0.39971023 -0.39388303 -0.47623784 ... -0.29193276  1.22485685
  -0.36495896]
 ...
 [-0.25578558 -0.24067785 -0.35419173 ... -0.23181242  0.75545779
  -0.27953781]
 [-0.32576602 -0.33141547 -0.37128933 ... -0.08775675  0.85471869
  -0.31608656]
 [-0.20729054 -0.26354787 -0.13859468 ... -0.05452214  0.66584304
  -0.15206914]]
y_out_last.shape:  (1024000, 512)
y_out_last_re.shape:  (1000, 32, 32, 512)
